In [1]:
import pyspark
import pandas as pd
from pca import pca
from typing import Union, List, Dict
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, BooleanType
from pyspark.sql.functions import pandas_udf, PandasUDFType, lit, col, when, sum as spark_sum, monotonically_increasing_id

In [2]:
import pyspark
import pandas as pd
from pca import pca
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from pyspark.sql.types import StructType, StructField, StringType, FloatType, BooleanType
from pyspark.sql.functions import pandas_udf, PandasUDFType, lit, col, countDistinct, when, monotonically_increasing_id, \
    sum as spark_sum
from typing import List, Dict, Union

In [3]:
import os
import json
from pyspark.sql import SparkSession
import pyspark.pandas as ps

os.environ['PYSPARK_PYTHON'] = '/usr/local/python-3.9.13/bin/python3'
spark = SparkSession.builder \
    .appName("pandas_udf") \
    .config('spark.sql.session.timeZone', 'Asia/Shanghai') \
    .config("spark.scheduler.mode", "FAIR") \
    .config('spark.driver.memory', '8g') \
    .config('spark.driver.cores', '12') \
    .config('spark.executor.memory', '8g') \
    .config('spark.executor.cores', '12') \
    .config('spark.cores.max', '12') \
    .config('spark.driver.host', '192.168.22.28') \
    .master("spark://192.168.12.47:7077,192.168.12.48:7077") \
    .getOrCreate()

[scatterd] >WARNING> 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.


In [4]:
10+7+53+73+53+219

415

In [5]:
df_pandas = pd.read_csv(
    "D:/Jupyterfiles/晶合MVAFDC_general开发/MVAanlysisDevelop/inline_algorithm/inline_case5_label.csv")
df_pandas = df_pandas[
    df_pandas['OPE_NO'].isin(['1V.EQW10', '1V.PQW10', '1F.FQE10', '1C.CDG10', '1U.EQW10', '1U.PQW10'])]
df_spark = ps.from_pandas(df_pandas).to_spark()
print(f"df_spark shape: ({df_spark.count()}, {len(df_spark.columns)})")
df_spark.show()

C:\Users\yang.wenjun\AppData\Local\Temp\ipykernel_25844\72344417.py:1: DtypeWarning: Columns (134) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pandas = pd.read_csv(
D:\developer\Anaconda\lib\site-packages\pyspark\pandas\internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
D:\developer\Anaconda\lib\site-packages\pyspark\sql\pandas\conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
D:\developer\Anaconda\lib\site-packages\pyspark\pandas\utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


df_spark shape: (2461, 144)
+---------+--------+-------------------+-------------------+-----------+------+--------------+---------+-----------------+---------+---------+----------+---------------+-------------+-------------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+--------------+--------+------------+-------+-------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+-------

In [62]:
oper = '1C.CDG10'
columns_list =  ['OPE_NO'] + ['WAFER_ID', 'INLINE_PARAMETER_ID', 'AVERAGE', 'MAX_VAL', 'MEDIAN', 'MIN_VAL',
                                 'STD_DEV', 'PERCENTILE_25', 'PERCENTILE_75', 'SITE_COUNT', 'label']
df_run1 = df_pandas.query(f"OPE_NO == '{oper}'")[columns_list]
df_run1[df_run1['INLINE_PARAMETER_ID']=='CIW0']

,OPE_NO,WAFER_ID,INLINE_PARAMETER_ID,AVERAGE,MAX_VAL,MEDIAN,MIN_VAL,STD_DEV,PERCENTILE_25,PERCENTILE_75,SITE_COUNT,label
13,1C.CDG10,NAZ703-08,CIW0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
5696,1C.CDG10,NAZ439-07,CIW0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7394,1C.CDG10,NAZ439-03,CIW0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9449,1C.CDG10,NBX082-05,CIW0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
11806,1C.CDG10,NAZ415-06,CIW0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
12835,1C.CDG10,NBX082-16,CIW0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
15671,1C.CDG10,NAZ439-06,CIW0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
17756,1C.CDG10,NBX082-12,CIW0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
22579,1C.CDG10,NAZ415-08,CIW0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
24221,1C.CDG10,NAZ415-12,CIW0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [7]:
df_pandas.groupby('OPE_NO')['INLINE_PARAMETER_ID'].nunique()

OPE_NO
1C.CDG10     11
1F.FQE10     15
1U.EQW10     97
1U.PQW10     74
1V.EQW10    117
1V.PQW10    270
Name: INLINE_PARAMETER_ID, dtype: int64

In [9]:
json_loads_dict = {"requestId": "269",
                   "algorithm": "inline_by_wafer",
                   "requestParam": {"dateRange": {"start": "2021-12-06 19:50:49", "end": "2024-03-06 19:50:49"},
                                    "operNo": [],
                                    "uploadId": "84f6a2b46a5443ec9797347424402058",
                                    "flagMergeAllProdg1": "0",
                                    "flagMergeAllProductId": "0",
                                    "flagMergeAllChamber": "0",
                                    "mergeProdg1": [],
                                    "mergeProductId": [],
                                    "mergeEqp": [],
                                    "mergeChamber": [],

                                    # "mergeOperno": [{"1F.FQE10,1C.CDG10": ["1F.FQE10", "1C.CDG10"]},
                                    #                 {"1U.EQW10_1U.PQW10": ["1U.EQW10", "1U.PQW10"]}],

                                    "mergeOperno": None
                                    }
                   }

df_ = pd.DataFrame({"requestId": [json_loads_dict["requestId"]],
                    "requestParam": [json.dumps(json_loads_dict["requestParam"])]})

request_id_ = df_["requestId"].values[0]
request_params = df_["requestParam"].values[0]
parse_dict = json.loads(request_params)

merge_operno = list(parse_dict.get('mergeOperno')) if parse_dict.get('mergeOperno') else None
merge_prodg1 = list(parse_dict.get('mergeProdg1')) if parse_dict.get('mergeProdg1') else None
merge_product = list(parse_dict.get('mergeProductId')) if parse_dict.get('mergeProductId') else None

In [53]:
class DataPreprocessorForInline:
    def __init__(self,
                 df: pyspark.sql.dataframe,
                 columns_list: list[str],
                 certain_column: str,
                 key_words: list[str],
                 convert_to_numeric_list: list[str],
                 merge_operno_list: List[Dict[str, List[str]]],
                 merge_prodg1_list: List[Dict[str, List[str]]],
                 merge_product_list: List[Dict[str, List[str]]]
                 ):
        self.df = df
        self.columns_list = columns_list
        self.certain_column = certain_column
        self.key_words = key_words
        self.convert_to_numeric_list = convert_to_numeric_list
        self.merge_operno_list = merge_operno_list
        self.merge_prodg1_list = merge_prodg1_list
        self.merge_product_list = merge_product_list

    @staticmethod
    def select_columns(df: pyspark.sql.dataframe, columns_list: list[str]) -> pyspark.sql.dataframe:
        return df.select(columns_list)

    @staticmethod
    def exclude_some_data(df: pyspark.sql.dataframe, key_words: list[str],
                          certain_column: str) -> pyspark.sql.dataframe:
        key_words_str = '|'.join(key_words)
        df_filtered = df.filter(~col(certain_column).rlike(key_words_str))
        return df_filtered

    @staticmethod
    def pre_process(df: pyspark.sql.dataframe, convert_to_numeric_list: list[str]) -> pyspark.sql.dataframe:
        for column in convert_to_numeric_list:
            df = df.withColumn(column, col(column).cast('double'))
        if 'SITE_COUNT' in convert_to_numeric_list:
            convert_to_numeric_list.remove('SITE_COUNT')
        df = df.dropna(subset=convert_to_numeric_list, how='all')
        return df

    @staticmethod
    def integrate_columns(df: pyspark.sql.dataframe,
                          merge_operno_list: List[Dict[str, List[str]]],
                          merge_prodg1_list: List[Dict[str, List[str]]],
                          merge_product_list: List[Dict[str, List[str]]]) -> pyspark.sql.dataframe:
        """
        Integrate columns in the DataFrame based on the provided list.

        :param df: The input DataFrame.
        :param merge_operno_list: A list of dictionaries where each dictionary contains values to be merged.
               Example: [{'2F.CDS10_XX.TDS01': ['2F.CDS10', 'XX.TDS01']},
                         {'2F.CDS20_XX.CDS20': ['2F.CDS20', 'XX.CDS20']}]
        :param merge_prodg1_list: A list of dictionaries for merging 'PRODG1' column in a similar fashion.
        :param merge_product_list: A list of dictionaries for merging 'PRODUCT_ID' column in a similar fashion.

        :return: DataFrame with 'OPER_NO' and other specified columns integrated according to the merge rules.
        """
        df_merged = DataPreprocessorForInline.integrate_single_column(df, merge_operno_list, 'OPER_NO')
        df_merged = DataPreprocessorForInline.integrate_single_column(df_merged, merge_prodg1_list, 'PRODG1')
        df_merged = DataPreprocessorForInline.integrate_single_column(df_merged, merge_product_list, 'PRODUCT_ID')
        return df_merged

    @staticmethod
    def integrate_single_column(df: pyspark.sql.dataframe,
                                merge_list: List[Dict[str, List[str]]],
                                column_name: str) -> pyspark.sql.dataframe:
        """
        Integrate columns in the DataFrame based on the provided list.

        :param df: The input DataFrame.
        :param merge_list: A list of dictionaries where each dictionary contains values to be merged.
        :param column_name: The name of the column to be merged.

        :return: DataFrame with specified column integrated according to the merge rules.
        """
        splitter_comma = ","
        if merge_list is not None and len(merge_list) > 0:
            values_to_replace = [list(rule.values())[0] for rule in merge_list]
            merged_values = [splitter_comma.join(list(rule.values())[0]) for rule in merge_list]

            for values, replacement_value in zip(values_to_replace, merged_values):
                df = df.withColumn(column_name,
                                   when(col(column_name).isin(values), replacement_value).otherwise(col(column_name)))
        return df

    def run(self) -> pyspark.sql.dataframe:
        df_select = self.select_columns(df=self.df, columns_list=self.columns_list)
        print("df_select:", df_select.count())
        df_esd = self.exclude_some_data(df=df_select, key_words=self.key_words, certain_column=self.certain_column)
        print("df_esd:", df_esd.count())
        df_pp = self.pre_process(df=df_esd, convert_to_numeric_list=self.convert_to_numeric_list)
        print("df_pp:", df_pp.count())
        df_integrate = self.integrate_columns(df=df_pp,
                                              merge_operno_list=self.merge_operno_list,
                                              merge_prodg1_list=self.merge_prodg1_list,
                                              merge_product_list=self.merge_product_list)
        print("df_integrate:", df_integrate.count())
        return df_integrate
    

class GetTrainDataForInline:
    def __init__(self, df: pyspark.sql.dataframe, grpby_list: list[str]):
        """
        Initialize the GetTrainDataForInline class.

        Parameters:
        - df (pyspark.sql.dataframe): Input DataFrame.
        - grpby_list (list): List of grouping columns.

        This class is designed to perform commonality analysis and retrieve training data based on the
        condition "bad_num >= 1 AND good_num >= 1" in each grpby_list, i.e. each OPE_NO for inline data.
        """
        self.df_run = df
        self.grpby_list = grpby_list

    @staticmethod
    def commonality_analysis(df_run: pyspark.sql.dataframe, grpby_list: list[str]) -> pyspark.sql.dataframe:
        grps_all = (df_run.groupBy(grpby_list)
                    .agg(countDistinct('WAFER_ID', when(df_run['label'] == 0, 1)).alias('good_num'),
                         countDistinct('WAFER_ID', when(df_run['label'] == 1, 1)).alias('bad_num'))
                    .na.fill(0))
        grps_all = grps_all.withColumn("conditions_satisfied",
                                       when((col('good_num') >= 2) & (col('bad_num') >= 6), True).otherwise(False))
        grps = grps_all.filter("good_num >= 2 AND bad_num >= 6")
        return grps_all, grps

    @staticmethod
    def get_data_list(common_res: pyspark.sql.dataframe, grpby_list: list[str]) -> List[Dict[str, str]]:
        data_list = common_res.select(grpby_list).collect()
        data_dict_list = [row.asDict() for row in data_list]
        return data_dict_list

    @staticmethod
    def get_train_data(df_run: pyspark.sql.dataframe, data_dict_list: List[Dict[str, str]]) -> pyspark.sql.dataframe:
        first_data_dict = data_dict_list[0]
        conditions = " AND ".join(["{} == '{}'".format(col_, first_data_dict[col_]) for col_ in first_data_dict])
        df_s = df_run.filter(conditions)
        for i in range(1, len(data_dict_list)):
            data_dict = data_dict_list[i]
            conditions = " AND ".join(["{} == '{}'".format(col_, data_dict[col_]) for col_ in data_dict])
            df_m = df_run.filter(conditions)
            df_s = df_s.union(df_m)
        return df_s
    
    def run(self) -> pyspark.sql.dataframe:
        grps_all, grps = self.commonality_analysis(df_run=self.df_run, grpby_list=self.grpby_list)
        grps_all.show()
        grps.show() 
        
        conditions_not_satisfied = grps_all.filter("conditions_satisfied==False").select(self.grpby_list + ["good_num", 'bad_num'])
        print("conditions_not_satisfied:")
        conditions_not_satisfied.show()
        
        parametric_for_unsatisfied_conditions = conditions_not_satisfied.join(self.df_run.select(self.grpby_list + ['INLINE_PARAMETER_ID']), self.grpby_list, "left").dropDuplicates()
        parametric_for_unsatisfied_conditions.show()
        
        if grps.isEmpty():
            msg = f"按照{'+'.join(self.grpby_list)}分组后的数据, 没有组合满足条件good >= 1且bad >= 1, 无法进行分析."
            raise RCABaseException(msg)

        if grps_all.count() == grps.count():
            return grps_all, self.df_run, parametric_for_unsatisfied_conditions
        else:
            data_dict_list = self.get_data_list(common_res=grps, grpby_list=self.grpby_list)
            print(data_dict_list)
            train_data = self.get_train_data(df_run=self.df_run, data_dict_list=data_dict_list)
            return grps_all, train_data, parametric_for_unsatisfied_conditions

In [54]:
grpby_list = ['OPER_NO']

columns_list = grpby_list + ['WAFER_ID', 'INLINE_PARAMETER_ID', 'AVERAGE', 'MAX_VAL', 'MEDIAN', 'MIN_VAL',
                                 'STD_DEV', 'PERCENTILE_25', 'PERCENTILE_75', 'SITE_COUNT', 'label']

key_words = ['CXS', 'CYS', 'FDS']

convert_to_numeric_list = ['AVERAGE', 'MAX_VAL', 'MEDIAN', 'MIN_VAL', 'STD_DEV', 'PERCENTILE_25',
                               'PERCENTILE_75', 'SITE_COUNT']

certain_column = 'INLINE_PARAMETER_ID'

df_spark = df_spark.withColumnRenamed('OPE_NO', 'OPER_NO')
df_preprocess = DataPreprocessorForInline(df=df_spark,
                                          columns_list=columns_list,
                                          certain_column=certain_column,
                                          key_words=key_words,
                                          convert_to_numeric_list=convert_to_numeric_list,
                                          merge_operno_list=merge_operno,
                                          merge_prodg1_list=merge_prodg1,
                                          merge_product_list=merge_product).run()
print(f"df_preprocess shape: ({df_preprocess.count()}, {len(df_preprocess.columns)})")
df_preprocess.show()

df_select: 2461
df_esd: 2451
df_pp: 2433
df_integrate: 2433
df_preprocess shape: (2433, 12)
+--------+---------+-------------------+----------------+--------+--------+--------+--------------+-------------+-------------+----------+-----+
| OPER_NO| WAFER_ID|INLINE_PARAMETER_ID|         AVERAGE| MAX_VAL|  MEDIAN| MIN_VAL|       STD_DEV|PERCENTILE_25|PERCENTILE_75|SITE_COUNT|label|
+--------+---------+-------------------+----------------+--------+--------+--------+--------------+-------------+-------------+----------+-----+
|1F.FQE10|NAZ439-03|               TAW1|        109.2519|    null|    null|    null|          null|         null|         null|      null|    0|
|1F.FQE10|NAZ439-03|               TAWB|        109.2519|    null|    null|    null|          null|         null|         null|      null|    0|
|1F.FQE10|NAZ439-03|               TDS0|        108.4117|108.4117|108.4117|108.4117|           0.0|     108.4117|     108.4117|       1.0|    0|
|1F.FQE10|NAZ439-03|               TDS

In [51]:
df_preprocess_pandas = df_preprocess.toPandas()

In [52]:
oper = '1C.CDG10'
df_run2 = df_preprocess_pandas.query(f"OPER_NO == '{oper}'") 
df_run2['INLINE_PARAMETER_ID'].value_counts()

EEW0    15
FEW0    15
HFT0    15
OEW0    15
PEW0    15
PTW0    15
REW0    15
SEW0    15
SFW0    15
TSW0    15
Name: INLINE_PARAMETER_ID, dtype: int64

In [12]:
grps_all, grps = GetTrainDataForInline.commonality_analysis(df_run=df_preprocess, grpby_list=grpby_list)

grps_all.show()
grps.show()

+--------+--------+-------+--------------------+
| OPER_NO|good_num|bad_num|conditions_satisfied|
+--------+--------+-------+--------------------+
|1U.PQW10|       2|      3|               false|
|1C.CDG10|       2|     13|                true|
|1V.EQW10|       2|      6|                true|
|1U.EQW10|       2|      4|               false|
|1V.PQW10|       2|      5|               false|
|1F.FQE10|       1|      1|               false|
+--------+--------+-------+--------------------+

+--------+--------+-------+--------------------+
| OPER_NO|good_num|bad_num|conditions_satisfied|
+--------+--------+-------+--------------------+
|1C.CDG10|       2|     13|                true|
|1V.EQW10|       2|      6|                true|
+--------+--------+-------+--------------------+



In [13]:
conditions_not_satisfied = grps_all.filter("conditions_satisfied==False").select(grpby_list + ["good_num", 'bad_num'])
print("conditions_not_satisfied:")
conditions_not_satisfied.show()

conditions_not_satisfied:
+--------+--------+-------+
| OPER_NO|good_num|bad_num|
+--------+--------+-------+
|1U.PQW10|       2|      3|
|1U.EQW10|       2|      4|
|1V.PQW10|       2|      5|
|1F.FQE10|       1|      1|
+--------+--------+-------+



In [14]:
parametric_for_unsatisfied_conditions = conditions_not_satisfied.join(df_preprocess.select(grpby_list + ['INLINE_PARAMETER_ID']), grpby_list, "left").dropDuplicates()
print(parametric_for_unsatisfied_conditions.count())
parametric_for_unsatisfied_conditions.show()

450
+--------+--------+-------+-------------------+
| OPER_NO|good_num|bad_num|INLINE_PARAMETER_ID|
+--------+--------+-------+-------------------+
|1U.PQW10|       2|      3|               YSIG|
|1U.PQW10|       2|      3|               XSIG|
|1U.PQW10|       2|      3|               DPW1|
|1U.PQW10|       2|      3|               DPW0|
|1U.PQW10|       2|      3|              DNW05|
|1U.PQW10|       2|      3|              DNW04|
|1U.PQW10|       2|      3|              DNW03|
|1U.PQW10|       2|      3|              DNW02|
|1U.PQW10|       2|      3|              DNW01|
|1U.PQW10|       2|      3|               65W0|
|1U.PQW10|       2|      3|               64W0|
|1U.PQW10|       2|      3|               63W0|
|1U.PQW10|       2|      3|               62W0|
|1U.PQW10|       2|      3|               61W0|
|1U.PQW10|       2|      3|               60W0|
|1U.PQW10|       2|      3|               59W0|
|1U.PQW10|       2|      3|               58W0|
|1U.PQW10|       2|      3|         

In [15]:
data_dict_list = GetTrainDataForInline.get_data_list(common_res=grps, grpby_list=grpby_list)
print(data_dict_list)

train_data = GetTrainDataForInline.get_train_data(df_run=df_preprocess, data_dict_list=data_dict_list)
print(train_data.count())
train_data.show()

[{'OPER_NO': '1C.CDG10'}, {'OPER_NO': '1V.EQW10'}]
708
+--------+---------+-------------------+------------+-------+------+-------+-------+-------------+-------------+----------+-----+
| OPER_NO| WAFER_ID|INLINE_PARAMETER_ID|     AVERAGE|MAX_VAL|MEDIAN|MIN_VAL|STD_DEV|PERCENTILE_25|PERCENTILE_75|SITE_COUNT|label|
+--------+---------+-------------------+------------+-------+------+-------+-------+-------------+-------------+----------+-----+
|1C.CDG10|NAZ703-08|               EEW0| 3999.795302|   null|  null|   null|   null|         null|         null|      null|    1|
|1C.CDG10|NAZ703-08|               FEW0|123.30957013|   null|  null|   null|   null|         null|         null|      null|    1|
|1C.CDG10|NAZ703-08|               HFT0|      44.599|   null|  null|   null|   null|         null|         null|      null|    1|
|1C.CDG10|NAZ703-08|               OEW0|5999.4883668|   null|  null|   null|   null|         null|         null|      null|    1|
|1C.CDG10|NAZ703-08|               

-----  

In [16]:
grps_all, df_train, parametric_for_unsatisfied_conditions = GetTrainDataForInline(df=df_preprocess, grpby_list=grpby_list).run()
print("grps_all:")
grps_all.show()
print(f"df_train shape: ({df_train.count()}, {len(df_train.columns)})")
df_train.show()
print("parametric_for_unsatisfied_conditions:", parametric_for_unsatisfied_conditions.count())
parametric_for_unsatisfied_conditions.show()

grps_all:
+--------+--------+-------+--------------------+
| OPER_NO|good_num|bad_num|conditions_satisfied|
+--------+--------+-------+--------------------+
|1U.PQW10|       2|      3|               false|
|1C.CDG10|       2|     13|                true|
|1V.EQW10|       2|      6|                true|
|1U.EQW10|       2|      4|               false|
|1V.PQW10|       2|      5|               false|
|1F.FQE10|       1|      1|               false|
+--------+--------+-------+--------------------+

df_train shape: (708, 12)
+--------+---------+-------------------+------------+-------+------+-------+-------+-------------+-------------+----------+-----+
| OPER_NO| WAFER_ID|INLINE_PARAMETER_ID|     AVERAGE|MAX_VAL|MEDIAN|MIN_VAL|STD_DEV|PERCENTILE_25|PERCENTILE_75|SITE_COUNT|label|
+--------+---------+-------------------+------------+-------+------+-------+-------+-------------+-------------+----------+-----+
|1C.CDG10|NAZ703-08|               EEW0| 3999.795302|   null|  null|   null|   null

In [20]:
df_train_pandas = df_train.toPandas()

In [21]:
df_train_pandas['OPER_NO'].value_counts()

1V.EQW10    558
1C.CDG10    150
Name: OPER_NO, dtype: int64

---- 

In [17]:
class FitInlineModelByWafer:
    def __init__(self,
                 df: pyspark.sql.dataframe,
                 grpby_list: list[str],
                 columns_to_process: list[str],
                 missing_value_threshold: float,
                 model: str = 'pca'):
        self.df = df
        self.grpby_list = grpby_list
        self.columns_to_process = columns_to_process
        self.missing_value_threshold = missing_value_threshold
        self.model = model

    @staticmethod
    def process_missing_values(df, columns_to_process, missing_value_threshold):
        for column in columns_to_process:
            missing_percentage = df[column].isnull().mean()
            if missing_percentage > missing_value_threshold:
                df = df.drop(columns=[column])
            else:
                df[column] = df[column].fillna(df[column].mean())
        return df

    @staticmethod
    def get_pivot_table(df, grpby_list, columns_to_process, missing_value_threshold):
        df_specific = FitInlineModelByWafer.process_missing_values(df, columns_to_process, missing_value_threshold)
        index_list = ['WAFER_ID', 'label']
        columns_list = grpby_list + ['INLINE_PARAMETER_ID']
        values_list = df_specific.columns.difference(
            ['WAFER_ID', 'INLINE_PARAMETER_ID', 'SITE_COUNT', 'label'] + grpby_list)
        pivot_result = df_specific.pivot_table(index=index_list,
                                               columns=columns_list,
                                               values=values_list)
        pivot_result.columns = pivot_result.columns.map('#'.join)
        pivot_result = FitInlineModelByWafer.process_missing_values(pivot_result, pivot_result.columns,
                                                                    missing_value_threshold)
        pivot_result = pivot_result.reset_index(drop=False)
        # Remove completely identical columns
        for column in pivot_result.columns.difference(index_list):
            if pivot_result[column].nunique() == 1:
                pivot_result = pivot_result.drop(column, axis=1)
        return pivot_result

    @staticmethod
    def fit_pca_model(df, grpby_list, columns_to_process, missing_value_threshold):
        schema_all = StructType([StructField("features", StringType(), True),
                                 StructField("importance", FloatType(), True),
                                 StructField("algorithm_satisfied", BooleanType(), True),
                                 StructField("x_train_shape", StringType(), True)])

        @pandas_udf(returnType=schema_all, functionType=PandasUDFType.GROUPED_MAP)
        def get_model_result(df_run):
            pivot_result = FitInlineModelByWafer.get_pivot_table(df=df_run,
                                                                 grpby_list=grpby_list,
                                                                 columns_to_process=columns_to_process,
                                                                 missing_value_threshold=missing_value_threshold)

            x_train = pivot_result[pivot_result.columns.difference(['WAFER_ID', 'label']).tolist()]

            if min(x_train.shape) > 9:
                res_top_select = FitInlineModelByWafer.construct_features_when_satisfy_pca(x_train=x_train)
                return res_top_select
            else:
                res_top_select = FitInlineModelByWafer.construct_features_when_not_satisfied(x_train=x_train)
                return res_top_select

        return df.groupby(grpby_list).apply(get_model_result)

    @staticmethod
    def construct_features_when_not_satisfied(x_train):
        x_len = len(x_train.columns)
        res_top_select = pd.DataFrame({"features": x_train.columns,
                                       "importance": [0.0] * x_len,
                                       "algorithm_satisfied": [False] * x_len,
                                       "x_train_shape": [str(x_train.shape)] * x_len})
        return res_top_select

    @staticmethod
    def construct_features_when_satisfy_pca(x_train):
        # 得到PCA算法结果res_top_select
        n_components = min(min(x_train.shape) - 2, 20)
        model = pca(n_components=n_components, verbose=None)
        results = model.fit_transform(x_train)
        res_top = results['topfeat']
        res_top_select = res_top[res_top['type'] == 'best'][['feature', 'loading']]
        res_top_select['importance'] = abs(res_top_select['loading'])
        res_top_select = res_top_select.rename(columns={'feature': 'features'}).drop("loading",
                                                                                     axis=1).drop_duplicates()
        # 增加算法为0的features
        has_importance_features = res_top_select['features'].values
        zero_importance_features = x_train.columns.difference(has_importance_features).to_list()
        len_f = len(zero_importance_features)
        zero_df  = pd.DataFrame({'features': zero_importance_features, 'importance': [0.0] * len_f})
        res_top_select = res_top_select.append(zero_df, ignore_index=True)
        
        # 合并二者的结果
        res_top_select['x_train_shape'] = str(x_train.shape)
        res_top_select['algorithm_satisfied'] = True
        return res_top_select

    def run(self):
        if self.model == 'pca':
            res = self.fit_pca_model(df=self.df, grpby_list=self.grpby_list,
                                     columns_to_process=self.columns_to_process,
                                     missing_value_threshold=self.missing_value_threshold)

        else:
            res = self.fit_classification_model(df=self.df, grpby_list=self.grpby_list,
                                                columns_to_process=self.columns_to_process,
                                                missing_value_threshold=self.missing_value_threshold,
                                                model=self.model)
        return res

In [18]:
res = FitInlineModelByWafer(df=df_train,
                            grpby_list=grpby_list,
                            columns_to_process=['AVERAGE', 'MAX_VAL', 'MEDIAN', 'MIN_VAL', 'STD_DEV',
                                                'PERCENTILE_25', 'PERCENTILE_75'],
                            missing_value_threshold=0.6,
                            model='pca').run()

m = res.filter('algorithm_satisfied==True').count()
print(f"res shape: ({res.count()}, {len(res.columns)})")
res.show(50)

D:\developer\Anaconda\lib\site-packages\pyspark\sql\pandas\group_ops.py:98: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


res shape: (64, 4)
+--------------------+----------+-------------------+-------------+
|            features|importance|algorithm_satisfied|x_train_shape|
+--------------------+----------+-------------------+-------------+
|AVERAGE#1C.CDG10#...| 0.9999995|               true|     (15, 10)|
|AVERAGE#1C.CDG10#...|  0.782774|               true|     (15, 10)|
|AVERAGE#1C.CDG10#...|0.77940005|               true|     (15, 10)|
|AVERAGE#1C.CDG10#...|0.94151205|               true|     (15, 10)|
|AVERAGE#1C.CDG10#...| 0.7007957|               true|     (15, 10)|
|AVERAGE#1C.CDG10#...|0.70067024|               true|     (15, 10)|
|AVERAGE#1C.CDG10#...|0.96751493|               true|     (15, 10)|
|AVERAGE#1C.CDG10#...| 0.9823262|               true|     (15, 10)|
|AVERAGE#1C.CDG10#...|       0.0|               true|     (15, 10)|
|AVERAGE#1C.CDG10#...|       0.0|               true|     (15, 10)|
|AVERAGE#1C.CDG10#...|       0.0|               true|     (15, 10)|
|AVERAGE#1V.EQW10#...|       

In [29]:
oper = '1C.CDG10'
df_run = df_train_pandas.query(f"OPER_NO == '{oper}'") 
df_run

,OPER_NO,WAFER_ID,INLINE_PARAMETER_ID,AVERAGE,MAX_VAL,MEDIAN,MIN_VAL,STD_DEV,PERCENTILE_25,PERCENTILE_75,SITE_COUNT,label
0,1C.CDG10,NAZ703-08,EEW0,3999.795302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,1C.CDG10,NAZ703-08,FEW0,123.309570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,1C.CDG10,NAZ703-08,HFT0,44.599000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,1C.CDG10,NAZ703-08,OEW0,5999.488367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,1C.CDG10,NAZ703-08,PEW0,6.999992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...
145,1C.CDG10,NAZ703-01,PTW0,44800.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
146,1C.CDG10,NAZ703-01,REW0,203.351449,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
147,1C.CDG10,NAZ703-01,SEW0,1.299785,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
148,1C.CDG10,NAZ703-01,SFW0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [36]:
df_run['INLINE_PARAMETER_ID'].value_counts()

EEW0    15
FEW0    15
HFT0    15
OEW0    15
PEW0    15
PTW0    15
REW0    15
SEW0    15
SFW0    15
TSW0    15
Name: INLINE_PARAMETER_ID, dtype: int64

In [30]:
index_list = ['WAFER_ID', 'label']
columns_list = grpby_list + ['INLINE_PARAMETER_ID']
values_list = df_run.columns.difference(
    ['WAFER_ID', 'INLINE_PARAMETER_ID', 'SITE_COUNT', 'label'] + grpby_list)
pivot_result = df_run.pivot_table(index=index_list,
                                       columns=columns_list,
                                       values=values_list)
pivot_result

AVERAGE                                             \
OPER_NO                 1C.CDG10                                              
INLINE_PARAMETER_ID         EEW0        FEW0    HFT0         OEW0      PEW0   
WAFER_ID  label                                                               
NAZ415-06 1          3999.845454  123.402563  44.099  6000.016364  6.999985   
NAZ415-08 1          3999.889091  123.401766  44.100  6000.001364  7.000001   
NAZ415-12 1          3999.932727  123.412397  44.099  6000.001364  7.000014   
NAZ415-13 1          3999.928636  123.673911  43.100  6000.060000  7.000024   
NAZ439-03 0          3999.706500  124.113900  41.699  5999.835300  7.000000   
NAZ439-06 1          3999.768300  123.573100  41.798  5999.833800  7.000000   
NAZ439-07 0          3999.735300  124.114300  41.699  5999.790600  7.000000   
NAZ703-01 1          3999.844966  123.357162  44.798  5999.457047  7.000002   
NAZ703-08 1          3999.795302  123.309570  44.599  5999.488367  6.999992   
NAZ703-09 1          3999.795302  123.100338  44.798  5999.532215  7.000021   
NBX082-05 1          3999.900000  123.234603  43.099  6000.070909  6.999988   
NBX082-12 1          3999.506977  123.711549  43.099  5999.965581  7.000024   
NBX082-15 1          3999.554419  123.716831  42.900  5999.902791  6.999964   
NBX082-16 1          3999.554419  123.439953  43.099  5999.902791  6.999964   
NBX219-17 1          3999.562791  123.692535  42.899  5999.973953  7.000008   

                                                                     
OPER_NO                                                              
INLINE_PARAMETER_ID     PTW0        REW0      SEW0 SFW0        TSW0  
WAFER_ID  label                                                      
NAZ415-06 1          44100.0  204.067595  1.300270  1.0  350.383182  
NAZ415-08 1          44100.0  203.997189  1.299888  2.0  350.331818  
NAZ415-12 1          44100.0  204.403759  1.299901  4.0  350.314545  
NAZ415-13 1          44100.0  204.037898  1.300469  5.0  350.330455  
NAZ439-03 0          41800.0  204.085700  1.300600  2.0  350.311800  
NAZ439-06 1          41800.0  199.861800  1.300200  3.0  350.281100  
NAZ439-07 0          41800.0  204.254800  1.300300  4.0  350.271900  
NAZ703-01 1          44800.0  203.351449  1.299785  1.0  350.371588  
NAZ703-08 1          44799.0  200.387121  1.299866  4.0  350.287919  
NAZ703-09 1          44800.0  203.449059  1.299604  5.0  350.373602  
NBX082-05 1          44100.0  204.704533  1.286355  3.0  350.367727  
NBX082-12 1          43100.0  203.236731  1.299651  3.0  350.246047  
NBX082-15 1          43100.0  204.264713  1.300241  1.0  350.327907  
NBX082-16 1          43100.0  203.184594  1.300241  1.0  350.330233  
NBX219-17 1          43100.0  204.825505  1.299845  3.0  350.320930

In [24]:
pivot_result = FitInlineModelByWafer.get_pivot_table(df=df_run,
                                                     grpby_list=grpby_list,
                                                     columns_to_process=['AVERAGE', 'MAX_VAL', 'MEDIAN', 'MIN_VAL', 'STD_DEV',
                                                                'PERCENTILE_25', 'PERCENTILE_75'],
                                                     missing_value_threshold=0.6)

x_train = pivot_result[pivot_result.columns.difference(['WAFER_ID', 'label']).tolist()]
x_train

C:\Users\yang.wenjun\AppData\Local\Temp\ipykernel_25844\4257316784.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].fillna(df[column].mean())


,AVERAGE#1V.EQW10#01W0,AVERAGE#1V.EQW10#02W0,AVERAGE#1V.EQW10#03W0,AVERAGE#1V.EQW10#04W0,AVERAGE#1V.EQW10#05W0,AVERAGE#1V.EQW10#06W0,AVERAGE#1V.EQW10#07W0,AVERAGE#1V.EQW10#08W0,AVERAGE#1V.EQW10#09W0,AVERAGE#1V.EQW10#10W0,...,AVERAGE#1V.EQW10#44W0,AVERAGE#1V.EQW10#45W0,AVERAGE#1V.EQW10#DNW01,AVERAGE#1V.EQW10#DNW02,AVERAGE#1V.EQW10#DNW03,AVERAGE#1V.EQW10#DNW04,AVERAGE#1V.EQW10#DNW05,AVERAGE#1V.EQW10#DNW06,AVERAGE#1V.EQW10#XSIG,AVERAGE#1V.EQW10#YSIG
0,0.0555,0.0598,0.0539,0.0519,0.0568,0.0577,0.052500,0.056200,0.0611,0.0600,...,0.0830,0.0811,0.0574,0.0545,0.0544,0.0849,0.0853,0.0523,0.0101,0.0066
1,0.0527,0.0521,0.0522,0.0525,0.0527,0.0514,0.052100,0.052900,0.0529,0.0534,...,0.0746,0.0727,0.0527,0.0502,0.0497,0.0742,0.0747,0.0523,0.0018,0.0023
2,0.0544,0.0594,0.0558,0.0523,0.0575,0.0568,0.053500,0.054600,0.0593,0.0577,...,0.0834,0.0797,0.0575,0.0548,0.0542,0.0847,0.0858,0.0523,0.0109,0.0087
3,0.0553,0.0525,0.0525,0.0785,0.0786,0.0528,0.052300,0.054100,0.0588,0.0561,...,0.0774,0.0786,0.0567,0.0545,0.0540,0.0837,0.0845,0.0526,0.0084,0.0083
4,0.0559,0.0528,0.0531,0.0773,0.0778,0.0489,0.050686,0.051529,0.0563,0.0532,...,0.0784,0.0792,0.0569,0.0549,0.0562,0.0808,0.0818,0.0523,0.0083,0.0071
5,0.0485,0.0494,0.0482,0.0473,0.0476,0.0482,0.049100,0.047300,0.0487,0.0495,...,0.0475,0.0474,0.0484,0.0471,0.0465,0.0698,0.0702,0.0516,0.0023,0.0018
6,0.0486,0.0486,0.0479,0.0483,0.0480,0.0489,0.048900,0.048800,0.0482,0.0495,...,0.0478,0.0476,0.0486,0.0471,0.0474,0.0670,0.0677,0.0527,0.0012,0.0012
7,0.0461,0.0467,0.0467,0.0459,0.0472,0.0462,0.046400,0.046800,0.0456,0.0464,...,0.0696,0.0676,0.0465,0.0477,0.0684,0.0481,0.0697,0.0523,0.0014,0.0016


In [97]:
# x_train = pd.DataFrame()
# FitInlineModelByWafer.construct_features_when_not_satisfied(x_train=x_train)

In [50]:
# n_components = min(min(x_train.shape) - 2, 20)
# model = pca(n_components=n_components, verbose=None)
# results = model.fit_transform(x_train)
# res_top = results['topfeat']
# res_top

In [59]:
res_top_select = FitInlineModelByWafer.construct_features_when_satisfy_pca(x_train=x_train)
res_top_select

,features,importance,x_train_shape,algorithm_satisfied
0,AVERAGE#1V.EQW10#29W0,0.292856,"(8, 53)",True
1,AVERAGE#1V.EQW10#40W0,0.434410,"(8, 53)",True
2,AVERAGE#1V.EQW10#DNW04,0.444564,"(8, 53)",True
3,AVERAGE#1V.EQW10#27W0,0.298047,"(8, 53)",True
4,AVERAGE#1V.EQW10#40W0,0.611230,"(8, 53)",True
5,AVERAGE#1V.EQW10#26W0,0.457991,"(8, 53)",True


In [98]:
# has_importance_features = res_top_select['features'].values
# has_importance_features

In [99]:
# zero_importance_features = x_train.columns.difference(has_importance_features).to_list()
# len_f = len(zero_importance_features)

# zero_df  = pd.DataFrame({'features': zero_importance_features,
#                             'importance': [0.0] * len_f})
# zero_df

In [100]:
# res_top_select.append(zero_df, ignore_index=True)

In [101]:
# res = res.withColumn("algorithm_satisfied", res["algorithm_satisfied"].cast(StringType()))
# res_pandas = res.toPandas()
# res_pandas

In [102]:
# res_pandas.tail(50)

----

In [152]:
class SplitInlineModelResults:
    def __init__(self, df: pyspark.sql.dataframe, grpby_list: List[str], request_id: str,
                 grps_all: pyspark.sql.dataframe, parametric_for_unsatisfied_conditions):
        self.df = df
        self.grpby_list = grpby_list
        self.request_id = request_id
        self.grps_all = grps_all
        self.parametric_for_unsatisfied_conditions = parametric_for_unsatisfied_conditions

    @staticmethod
    def split_features(df: pd.DataFrame, index: int) -> str:
        return df['features'].apply(lambda x: x.split('#')[index])

    @staticmethod
    def get_split_features(df: pd.DataFrame, grpby_list: List[str]) -> pd.DataFrame:
        n_feats = len(grpby_list)
        for i in range(n_feats):
            df[grpby_list[i]] = SplitInlineModelResults.split_features(df, i + 1)

        df['INLINE_PARAMETER_ID'] = SplitInlineModelResults.split_features(df, n_feats + 1)
        df = df.drop(['features'], axis=1).reset_index(drop=True)
        return df

    @staticmethod
    def split_calculate_features(df: pyspark.sql.dataframe, grpby_list: List[str], by: str) -> pyspark.sql.dataframe:
        struct_fields = [StructField(col_, StringType(), True) for col_ in grpby_list]
        struct_fields.extend([StructField("INLINE_PARAMETER_ID", StringType(), True),
                              StructField("importance", FloatType(), True),
                              StructField("algorithm_satisfied", BooleanType(), True)])
        schema_all = StructType(struct_fields)

        @pandas_udf(returnType=schema_all, functionType=PandasUDFType.GROUPED_MAP)
        def get_model_result(df_run):
            split_table = SplitInlineModelResults.get_split_features(df=df_run, grpby_list=grpby_list)
            split_table_grpby = split_table.groupby(grpby_list + ['INLINE_PARAMETER_ID', 'algorithm_satisfied'])[
                'importance'].sum().reset_index(drop=False)
            return split_table_grpby

        return df.groupby(by).apply(get_model_result)

    @staticmethod
    def add_certain_column(df: pyspark.sql.dataframe, request_id: str, grpby_list, parametric_for_unsatisfied_conditions):
        total_importance = df.select(spark_sum("importance")).collect()[0][0]
        df = (df.withColumn("weight", col("importance") / total_importance)
              .withColumn("weight_percent", col("weight") * 100))

        df = df.drop('importance', 'algorithm_satisfied')
        
        if not parametric_for_unsatisfied_conditions.isEmpty():
            df = df.unionByName(parametric_for_unsatisfied_conditions, allowMissingColumns=True)

        df = df.orderBy(col("weight").desc())
        df = (df.withColumn("good_num", df["good_num"].cast(FloatType()))
              .withColumn("bad_num", df["bad_num"].cast(FloatType()))
              .withColumn('index_no', monotonically_increasing_id() + 1)
              .withColumn("request_id", lit(request_id))
              .withColumn("AVG_SPEC_CHK_RESULT_COUNT", lit(0.0)))

        info_list = ['PRODUCT_ID', 'OPER_NO', 'PRODG1']
        for column in info_list:
            if column not in df.columns:
                df = df.withColumn(column, lit(None).cast(StringType()))
        return df

    def run(self):
        df = self.df.withColumn('temp', lit(0))
        split_res = self.split_calculate_features(df=df, grpby_list=self.grpby_list, by='temp')
        print("split_res:", split_res.count())
        split_res.show()

        grps_all_new = self.grps_all.filter("conditions_satisfied==True").select(self.grpby_list + ['good_num', 'bad_num'])
        print("grps_all_new:")
        grps_all_new.show()
        split_res_update = split_res.join(grps_all_new, on=self.grpby_list, how='left')
        print("split_res_update:", split_res_update.count())
        split_res_update.show()
        
        final_res = self.add_certain_column(df=split_res_update,
                                            request_id=self.request_id,
                                            grpby_list=self.grpby_list,
                                            parametric_for_unsatisfied_conditions=self.parametric_for_unsatisfied_conditions)
        print("final_res:", final_res.count())
        final_res.show()
        
        return final_res

In [153]:
final_res = SplitInlineModelResults(df=res, grpby_list=grpby_list, request_id='inline',
                                    grps_all=grps_all, parametric_for_unsatisfied_conditions=parametric_for_unsatisfied_conditions).run()

D:\developer\Anaconda\lib\site-packages\pyspark\sql\pandas\group_ops.py:98: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


split_res: 63
+--------+-------------------+----------+-------------------+
| OPER_NO|INLINE_PARAMETER_ID|importance|algorithm_satisfied|
+--------+-------------------+----------+-------------------+
|1C.CDG10|               EEW0|0.96751493|               true|
|1C.CDG10|               FEW0| 1.4014659|               true|
|1C.CDG10|               HFT0|0.94151205|               true|
|1C.CDG10|               OEW0|       0.0|               true|
|1C.CDG10|               PEW0|       0.0|               true|
|1C.CDG10|               PTW0| 0.9999995|               true|
|1C.CDG10|               REW0|  0.782774|               true|
|1C.CDG10|               SEW0|       0.0|               true|
|1C.CDG10|               SFW0|0.77940005|               true|
|1C.CDG10|               TSW0| 0.9823262|               true|
|1V.EQW10|               01W0|       0.0|              false|
|1V.EQW10|               02W0|       0.0|              false|
|1V.EQW10|               03W0|       0.0|              f

In [134]:
final_res = final_res.withColumn("algorithm_satisfied", final_res["algorithm_satisfied"].cast(StringType()))
final_res_pandas = final_res.toPandas()

final_res_pandas.to_csv('fff.csv')